In [1]:
"""
============================================================
  AWS SageMaker - Liver Disease Prediction Pipeline
  Assignment: End-to-End ML with SageMaker Components
  Dataset: https://www.kaggle.com/datasets/abhi8923shriv/liver-disease-patient-dataset
============================================================

Components Covered:
  1. Data Ingestion & Preprocessing
  2. SageMaker Feature Store
  3. SageMaker Pipelines (Data Pipeline)
  4. Model Training & Comparison (XGBoost vs Random Forest)
  5. CI/CD with SageMaker Pipelines + CodePipeline
  6. Model Deployment (Real-Time Endpoint)
  7. Model Monitoring (Data Quality + Model Quality)
"""

# ─────────────────────────────────────────────────────────
# 0.  IMPORTS & CONFIGURATION
# ─────────────────────────────────────────────────────────
import os, time, json, boto3, sagemaker
import pandas as pd
import numpy as np
from datetime import datetime

from sagemaker import get_execution_role, Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import (
    FeatureDefinition, FeatureTypeEnum
)

# SageMaker Pipelines
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, TrainingStep, TransformStep
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.parameters import (
    ParameterInteger, ParameterFloat, ParameterString
)
from sagemaker.workflow.properties import PropertyFile

# Processing & Training
from sagemaker.processing import (
    ProcessingInput, ProcessingOutput, ScriptProcessor
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker.xgboost.estimator import XGBoost  # kept for type hints only
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import (
    HyperparameterTuner, IntegerParameter, ContinuousParameter
)

# Model Registry & Deployment
from sagemaker.model import Model
from sagemaker.model_monitor import (
    DataCaptureConfig, DefaultModelMonitor,
    CronExpressionGenerator, ModelQualityMonitor
)
from sagemaker.model_monitor.dataset_format import DatasetFormat

# ── AWS / SageMaker Session ──────────────────────────────
boto_session   = boto3.Session(region_name="us-east-1")
sm_client      = boto_session.client("sagemaker")
s3_client      = boto_session.client("s3")

sagemaker_session = Session(boto_session=boto_session)
role              = get_execution_role()
region            = sagemaker_session.boto_region_name

BUCKET      = sagemaker_session.default_bucket()
PREFIX      = "liver-disease"
DATA_PREFIX = f"s3://{BUCKET}/{PREFIX}/data"
OUTPUT_PREFIX = f"s3://{BUCKET}/{PREFIX}/output"

print(f"Role    : {role}")
print(f"Bucket  : {BUCKET}")
print(f"Region  : {region}")


# ─────────────────────────────────────────────────────────
# 1.  DATA INGESTION – Upload raw CSV to S3
# ─────────────────────────────────────────────────────────
def upload_raw_data(local_csv_path: str = "Liver_Patient_Dataset__LPD__train.csv"):
    """
    Download dataset:
      Liver_Patient_Dataset__LPD__train.csv
    """
    s3_uri = sagemaker_session.upload_data(
        path=local_csv_path,
        bucket=BUCKET,
        key_prefix=f"{PREFIX}/data/raw"
    )
    print(f"Raw data uploaded → {s3_uri}")
    return s3_uri


# ─────────────────────────────────────────────────────────
# 2.  PREPROCESSING SCRIPT  (saved to disk, run via Pipeline)
# ─────────────────────────────────────────────────────────
PREPROCESSING_SCRIPT = """
import pandas as pd
import numpy as np
import os, argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-data",      type=str, default="/opt/ml/processing/input")
    parser.add_argument("--train-output",    type=str, default="/opt/ml/processing/train")
    parser.add_argument("--test-output",     type=str, default="/opt/ml/processing/test")
    parser.add_argument("--baseline-output", type=str, default="/opt/ml/processing/baseline")
    args = parser.parse_args()

    # ── Load ──────────────────────────────────────────────
    csv_file = os.path.join(args.input_data, "Liver_Patient_Dataset__LPD__train.csv")
    df = pd.read_csv(csv_file, encoding="latin-1")
    print(f"Loaded {len(df)} rows, {df.shape[1]} cols")

    # ── Strip invisible chars from column names (\\xa0 prefix) ──
    df.columns = df.columns.str.strip().str.replace("\\xa0", "", regex=False)

    # ── Rename columns to clean short names ───────────────
    df.rename(columns={
        "Age of the patient":                  "Age",
        "Gender of the patient":               "Gender",
        "Total Bilirubin":                     "Total_Bilirubin",
        "Direct Bilirubin":                    "Direct_Bilirubin",
        "Alkphos Alkaline Phosphotase":        "Alkaline_Phosphotase",
        "Sgpt Alamine Aminotransferase":       "Alamine_Aminotransferase",
        "Sgot Aspartate Aminotransferase":     "Aspartate_Aminotransferase",
        "Total Protiens":                      "Total_Proteins",
        "ALB Albumin":                         "Albumin",
        "A/G Ratio Albumin and Globulin Ratio": "AG_Ratio",
        "Result":                              "Target"
    }, inplace=True)

    # ── Clean ─────────────────────────────────────────────
    df.dropna(inplace=True)
    df["Gender"] = (df["Gender"] == "Male").astype(int)
    # Target: 1 = liver disease, 0 = no disease  (original: 1=disease, 2=no disease)
    df["Target"] = (df["Target"] == 1).astype(int)

    # ── Feature Engineering ───────────────────────────────
    df["Bilirubin_ratio"] = df["Direct_Bilirubin"] / (df["Total_Bilirubin"] + 1e-6)
    df["Enzyme_ratio"]    = (df["Alamine_Aminotransferase"] /
                             (df["Aspartate_Aminotransferase"] + 1e-6))
    df["Age_group"]       = pd.cut(df["Age"],
                                   bins=[0, 30, 50, 70, 120],
                                   labels=[0, 1, 2, 3]).astype(int)

    print(f"Class distribution: {df['Target'].value_counts().to_dict()}")

    # ── Split ─────────────────────────────────────────────
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler

    feature_cols = [c for c in df.columns if c != "Target"]
    X = df[feature_cols]
    y = df["Target"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train_sc = pd.DataFrame(scaler.fit_transform(X_train), columns=feature_cols)
    X_test_sc  = pd.DataFrame(scaler.transform(X_test),      columns=feature_cols)

    # ── Save ──────────────────────────────────────────────
    os.makedirs(args.train_output,    exist_ok=True)
    os.makedirs(args.test_output,     exist_ok=True)
    os.makedirs(args.baseline_output, exist_ok=True)

    train_df = pd.concat([y_train.reset_index(drop=True),
                          X_train_sc.reset_index(drop=True)], axis=1)
    test_df  = pd.concat([y_test.reset_index(drop=True),
                          X_test_sc.reset_index(drop=True)], axis=1)

    train_df.to_csv(os.path.join(args.train_output,    "train.csv"),    index=False, header=False)
    test_df.to_csv(os.path.join(args.test_output,      "test.csv"),     index=False, header=False)
    test_df.to_csv(os.path.join(args.baseline_output,  "baseline.csv"), index=False)   # with header for Monitor

    print(f"Train: {len(train_df)}, Test: {len(test_df)}")
    print("Preprocessing complete.")
"""

def save_preprocessing_script(path="preprocessing.py"):
    with open(path, "w") as f:
        f.write(PREPROCESSING_SCRIPT)
    print(f"Preprocessing script saved → {path}")


# ─────────────────────────────────────────────────────────
# 3.  SAGEMAKER FEATURE STORE
# ─────────────────────────────────────────────────────────
FEATURE_GROUP_NAME = "liver-disease-features"

def create_feature_group(df: pd.DataFrame):
    """
    Ingest features into SageMaker Feature Store.
    Requires two mandatory columns: record_id (String) and event_time (String).
    """
    # Add required FS columns
    df = df.copy()
    df["record_id"]  = df.index.astype(str)
    df["event_time"] = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
    df["Gender"]     = df["Gender"].astype(str)   # FS needs consistent types

    # ── Feature Definitions ───────────────────────────────
    feature_defs = []
    type_map = {
        "object":  FeatureTypeEnum.STRING,
        "float64": FeatureTypeEnum.FRACTIONAL,
        "int64":   FeatureTypeEnum.INTEGRAL,
    }
    for col in df.columns:
        dtype = str(df[col].dtype)
        fs_type = type_map.get(dtype, FeatureTypeEnum.STRING)
        feature_defs.append(FeatureDefinition(feature_name=col, feature_type=fs_type))

    # ── Create Feature Group ──────────────────────────────
    fg = FeatureGroup(
        name=FEATURE_GROUP_NAME,
        sagemaker_session=sagemaker_session,
        feature_definitions=feature_defs
    )

    # ── Check if feature group already exists ─────────────
    try:
        existing = sm_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP_NAME)
        existing_status = existing["FeatureGroupStatus"]
        print(f"Feature group '{FEATURE_GROUP_NAME}' already exists (status: {existing_status}). Skipping creation.")
    except sm_client.exceptions.ResourceNotFound:
        # Does not exist — create it
        fg.create(
            s3_uri=f"s3://{BUCKET}/{PREFIX}/feature-store",
            record_identifier_name="record_id",
            event_time_feature_name="event_time",
            role_arn=role,
            enable_online_store=True,
            description="Liver disease patient features"
        )
        print(f"Feature group '{FEATURE_GROUP_NAME}' creation initiated …")

        # Wait until Active
        status = ""
        while status != "Created":
            status = sm_client.describe_feature_group(
                FeatureGroupName=FEATURE_GROUP_NAME
            )["FeatureGroupStatus"]
            print(f"  Status: {status}")
            time.sleep(5)

    # ── Ingest Data ───────────────────────────────────────
    fg.ingest(data_frame=df, max_workers=5, wait=True)
    print(f"Ingested {len(df)} records into Feature Store.")
    return fg


def query_feature_store():
    """Run an Athena query against the offline Feature Store."""
    fg_desc = sm_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP_NAME)
    table   = fg_desc["OfflineStoreConfig"]["DataCatalogConfig"]["TableName"]
    db      = fg_desc["OfflineStoreConfig"]["DataCatalogConfig"]["Database"]

    query = f"""
        SELECT age, gender, total_bilirubin, dataset
        FROM "{db}"."{table}"
        WHERE dataset = '1'
        LIMIT 10
    """
    from sagemaker.feature_store.feature_group import FeatureGroup
    fg = FeatureGroup(name=FEATURE_GROUP_NAME, sagemaker_session=sagemaker_session)
    result = fg.athena_query()
    result.run(query_string=query,
               output_location=f"s3://{BUCKET}/{PREFIX}/athena-results/")
    result.wait()
    return result.as_dataframe()


# ─────────────────────────────────────────────────────────
# 4.  SAGEMAKER PIPELINE  (Data + Training + Evaluation)
# ─────────────────────────────────────────────────────────

# Pipeline Parameters (configurable at run-time)
pipeline_params = dict(
    processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1),
    training_instance_type    = ParameterString(name="TrainingInstanceType",     default_value="ml.m5.xlarge"),
    model_approval_status     = ParameterString(name="ModelApprovalStatus",      default_value="PendingManualApproval"),
    accuracy_threshold        = ParameterFloat(name="AccuracyThreshold",         default_value=0.75),
)


def build_pipeline() -> Pipeline:
    """
    Full SageMaker Pipeline:
      Step 1 – Preprocessing (SKLearn Processor)
      Step 2 – XGBoost Training  ← built-in algorithm (no custom script)
      Step 3 – RF Training       (SKLearn Estimator)
      Step 4 – Evaluation
      Step 5 – Conditional Registration (if accuracy ≥ threshold)
    """
    save_preprocessing_script()   # preprocessing.py still needed for Step 1

    # ── Step 1 : Preprocessing ────────────────────────────
    sklearn_processor = SKLearnProcessor(
        framework_version="1.0-1",
        instance_type="ml.m5.large",
        instance_count=pipeline_params["processing_instance_count"],
        role=role,
        sagemaker_session=sagemaker_session
    )

    step_process = ProcessingStep(
        name="LiverDiseasePreprocessing",
        processor=sklearn_processor,
        inputs=[
            ProcessingInput(
                source=f"s3://{BUCKET}/{PREFIX}/data/raw",
                destination="/opt/ml/processing/input"
            )
        ],
        outputs=[
            ProcessingOutput(output_name="train",    source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="test",     source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline"),
        ],
        code="preprocessing.py"
    )

    # ── Step 2 : XGBoost Training (built-in algorithm — no custom script) ──
    # Using the SageMaker built-in XGBoost image avoids all entry_point/module
    # issues. The container reads CSV directly; hyperparameters are passed as-is.
    xgb_image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=region,
        version="1.5-1"
    )

    xgb_estimator = Estimator(
        image_uri=xgb_image_uri,
        instance_type=pipeline_params["training_instance_type"],
        instance_count=1,
        role=role,
        output_path=f"{OUTPUT_PREFIX}/xgboost",
        sagemaker_session=sagemaker_session,
        hyperparameters={
            "max_depth":   5,
            "eta":         0.2,
            "subsample":   0.8,
            "num_round":   100,
            "objective":   "binary:logistic",
            "eval_metric": "auc",
        }
    )

    step_train_xgb = TrainingStep(
        name="TrainXGBoost",
        estimator=xgb_estimator,
        inputs={
            # Built-in XGBoost reads 'train' and 'validation' channel names
            "train": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig
                        .Outputs["train"].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig
                        .Outputs["test"].S3Output.S3Uri,
                content_type="text/csv"
            ),
        },
        depends_on=[step_process]
    )

    # ── Step 3 : Random Forest Training ───────────────────
    rf_estimator = SKLearn(
        entry_point="rf_train.py",        # created below
        framework_version="1.0-1",
        instance_type="ml.m5.xlarge",
        instance_count=1,
        role=role,
        output_path=f"{OUTPUT_PREFIX}/random-forest",
        sagemaker_session=sagemaker_session,
        hyperparameters={
            "n_estimators": 200,
            "max_depth":    10,
        }
    )

    step_train_rf = TrainingStep(
        name="TrainRandomForest",
        estimator=rf_estimator,
        inputs={
            "train": TrainingInput(
                s3_data=step_process.properties.ProcessingOutputConfig
                        .Outputs["train"].S3Output.S3Uri,
                content_type="text/csv"
            ),
        },
        depends_on=[step_process]
    )

    # ── Step 4 : Evaluation ───────────────────────────────
    eval_processor = SKLearnProcessor(
        framework_version="1.0-1",
        instance_type="ml.m5.large",
        instance_count=1,
        role=role,
        sagemaker_session=sagemaker_session
    )

    evaluation_report = PropertyFile(
        name="EvaluationReport",
        output_name="evaluation",
        path="evaluation.json"
    )

    step_eval = ProcessingStep(
        name="EvaluateModels",
        processor=eval_processor,
        inputs=[
            ProcessingInput(
                source=step_train_xgb.properties.ModelArtifacts.S3ModelArtifacts,
                destination="/opt/ml/processing/model/xgboost"
            ),
            ProcessingInput(
                source=step_train_rf.properties.ModelArtifacts.S3ModelArtifacts,
                destination="/opt/ml/processing/model/rf"
            ),
            ProcessingInput(
                source=step_process.properties.ProcessingOutputConfig
                       .Outputs["test"].S3Output.S3Uri,
                destination="/opt/ml/processing/test"
            ),
        ],
        outputs=[
            ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")
        ],
        code="evaluate.py",
        property_files=[evaluation_report]
    )

    # ── Step 5 : Conditional Model Registration ───────────
    # Register only if accuracy ≥ threshold
    step_register = RegisterModel(
        name="RegisterLiverDiseaseModel",
        estimator=xgb_estimator,
        model_data=step_train_xgb.properties.ModelArtifacts.S3ModelArtifacts,
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        transform_instances=["ml.m5.large"],
        model_package_group_name="LiverDiseaseModelGroup",
        approval_status=pipeline_params["model_approval_status"],
        description="XGBoost liver disease model registered via pipeline"
    )

    condition_acc = ConditionGreaterThanOrEqualTo(
        left=JsonGet(
            step_name=step_eval.name,
            property_file=evaluation_report,
            json_path="xgboost.accuracy"
        ),
        right=pipeline_params["accuracy_threshold"]
    )

    step_cond = ConditionStep(
        name="CheckAccuracy",
        conditions=[condition_acc],
        if_steps=[step_register],
        else_steps=[]
    )

    # ── Assemble Pipeline ─────────────────────────────────
    pipeline = Pipeline(
        name="LiverDiseasePipeline",
        parameters=list(pipeline_params.values()),
        steps=[step_process, step_train_xgb, step_train_rf, step_eval, step_cond],
        sagemaker_session=sagemaker_session
    )

    # Force-delete the old pipeline so the cached definition is fully replaced.
    # upsert() alone does not guarantee the execution uses the new step definitions.
    try:
        sm_client.delete_pipeline(PipelineName="LiverDiseasePipeline")
        print("Old pipeline deleted — recreating with updated definition …")
        time.sleep(3)   # brief pause for deletion to propagate
    except sm_client.exceptions.ResourceNotFound:
        pass   # doesn't exist yet, nothing to delete

    pipeline.upsert(role_arn=role)
    print("✅ Pipeline 'LiverDiseasePipeline' created with built-in XGBoost algorithm.")
    return pipeline


# ─────────────────────────────────────────────────────────
# 5.  TRAINING SCRIPTS  (written to disk before pipeline runs)
# ─────────────────────────────────────────────────────────

XGB_TRAIN_SCRIPT = """
import argparse, os, glob, traceback
import pandas as pd, numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score

def find_csv(directory):
    \"\"\"Find the first CSV in a directory regardless of filename.\"\"\"
    files = glob.glob(os.path.join(directory, "*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV found in {directory}. Contents: {os.listdir(directory)}")
    print(f"  Found data file: {files[0]}")
    return files[0]

if __name__ == "__main__":
    try:
        parser = argparse.ArgumentParser()
        parser.add_argument("--max_depth",   type=int,   default=5)
        parser.add_argument("--eta",         type=float, default=0.2)
        parser.add_argument("--subsample",   type=float, default=0.8)
        parser.add_argument("--num_round",   type=int,   default=100)
        parser.add_argument("--objective",   type=str,   default="binary:logistic")
        parser.add_argument("--eval_metric", type=str,   default="auc")
        parser.add_argument("--model-dir",   type=str,   default=os.environ.get("SM_MODEL_DIR", "/opt/ml/model"))
        parser.add_argument("--train",       type=str,   default=os.environ.get("SM_CHANNEL_TRAIN"))
        parser.add_argument("--test",        type=str,   default=os.environ.get("SM_CHANNEL_TEST"))
        args = parser.parse_args()

        print(f"Args: {args}")
        print(f"Train dir: {args.train}  contents: {os.listdir(args.train) if args.train else 'None'}")
        if args.test:
            print(f"Test  dir: {args.test}   contents: {os.listdir(args.test)}")

        # ── Load training data ────────────────────────────────
        train = pd.read_csv(find_csv(args.train), header=None)
        print(f"Train shape: {train.shape}")

        X_tr = train.iloc[:, 1:].values.astype(float)
        y_tr = train.iloc[:, 0].values.astype(float)
        dtrain = xgb.DMatrix(X_tr, label=y_tr)

        # ── Load test data (optional – used for early stopping) ──
        evals = [(dtrain, "train")]
        if args.test and os.path.isdir(args.test):
            test  = pd.read_csv(find_csv(args.test), header=None)
            print(f"Test shape: {test.shape}")
            X_te  = test.iloc[:, 1:].values.astype(float)
            y_te  = test.iloc[:, 0].values.astype(float)
            dtest = xgb.DMatrix(X_te, label=y_te)
            evals.append((dtest, "test"))
        else:
            print("No test channel found — training without early stopping.")
            dtest = None

        # ── Train ─────────────────────────────────────────────
        params = {
            "max_depth":   args.max_depth,
            "eta":         args.eta,
            "subsample":   args.subsample,
            "objective":   args.objective,
            "eval_metric": args.eval_metric,
            "seed":        42,
        }
        print(f"Training params: {params}")

        callbacks = []
        if dtest is not None:
            callbacks = [xgb.callback.EarlyStopping(rounds=10)]

        model = xgb.train(
            params, dtrain, args.num_round,
            evals=evals,
            callbacks=callbacks,
            verbose_eval=10
        )

        # ── Evaluate ──────────────────────────────────────────
        if dtest is not None:
            preds = (model.predict(dtest) >= 0.5).astype(int)
            acc   = accuracy_score(y_te, preds)
            auc   = roc_auc_score(y_te, model.predict(dtest))
            print(f"XGBoost → Accuracy: {acc:.4f}  AUC: {auc:.4f}")

        # ── Save ──────────────────────────────────────────────
        os.makedirs(args.model_dir, exist_ok=True)
        out_path = os.path.join(args.model_dir, "xgboost-model")
        model.save_model(out_path)
        print(f"Model saved to {out_path}")

    except Exception as e:
        print("=" * 60)
        print(f"TRAINING FAILED: {e}")
        traceback.print_exc()
        print("=" * 60)
        raise   # re-raise so SageMaker marks the job as failed
"""

RF_TRAIN_SCRIPT = """
import argparse, os, glob, traceback, joblib
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def find_csv(directory):
    files = glob.glob(os.path.join(directory, "*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV found in {directory}. Contents: {os.listdir(directory)}")
    print(f"  Found data file: {files[0]}")
    return files[0]

if __name__ == "__main__":
    try:
        parser = argparse.ArgumentParser()
        parser.add_argument("--n_estimators", type=int, default=200)
        parser.add_argument("--max_depth",    type=int, default=10)
        parser.add_argument("--model-dir",    type=str, default=os.environ.get("SM_MODEL_DIR", "/opt/ml/model"))
        parser.add_argument("--train",        type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
        args = parser.parse_args()

        print(f"Args: {args}")
        print(f"Train dir: {args.train}  contents: {os.listdir(args.train) if args.train else 'None'}")

        data = pd.read_csv(find_csv(args.train), header=None)
        print(f"Data shape: {data.shape}")

        X = data.iloc[:, 1:].values.astype(float)
        y = data.iloc[:, 0].values.astype(float)

        clf = RandomForestClassifier(
            n_estimators=args.n_estimators,
            max_depth=args.max_depth,
            random_state=42, n_jobs=-1
        )
        clf.fit(X, y)

        acc = accuracy_score(y, clf.predict(X))
        print(f"Random Forest → Train Accuracy: {acc:.4f}")

        os.makedirs(args.model_dir, exist_ok=True)
        out_path = os.path.join(args.model_dir, "rf_model.joblib")
        joblib.dump(clf, out_path)
        print(f"Model saved to {out_path}")

    except Exception as e:
        print("=" * 60)
        print(f"TRAINING FAILED: {e}")
        traceback.print_exc()
        print("=" * 60)
        raise
"""

EVALUATE_SCRIPT = """
import subprocess, sys

# Install xgboost at runtime — it is NOT pre-installed in the SKLearn processing container
subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost==1.5.2", "--quiet"])

import os, json, tarfile, glob, traceback
import pandas as pd, numpy as np
import joblib
import xgboost as xgb
from sklearn.metrics import (
    accuracy_score, roc_auc_score,
    precision_score, recall_score, f1_score
)

def list_dir(path):
    \"\"\"Print directory contents for debugging.\"\"\"
    if os.path.isdir(path):
        print(f"  Contents of {path}: {os.listdir(path)}")
    else:
        print(f"  Path does not exist: {path}")

def extract_tar(model_dir):
    \"\"\"Extract model.tar.gz and return the extracted directory path.\"\"\"
    # SageMaker puts the model artifact at either:
    #   <dir>/model.tar.gz          (S3 single-file download)
    #   <dir>/<job-name>/model.tar.gz  (rare, nested)
    tars = glob.glob(os.path.join(model_dir, "**", "model.tar.gz"), recursive=True)
    if not tars:
        raise FileNotFoundError(f"No model.tar.gz found under {model_dir}")
    tar_path = tars[0]
    extract_dir = os.path.dirname(tar_path)
    print(f"  Extracting {tar_path} → {extract_dir}")
    with tarfile.open(tar_path) as t:
        t.extractall(extract_dir)
    print(f"  After extract: {os.listdir(extract_dir)}")
    return extract_dir

def load_xgb(model_dir):
    extract_dir = extract_tar(model_dir)
    # Built-in XGBoost saves as 'xgboost-model'
    candidates = glob.glob(os.path.join(extract_dir, "xgboost-model*"))
    if not candidates:
        raise FileNotFoundError(f"No xgboost-model file found in {extract_dir}")
    model_file = candidates[0]
    print(f"  Loading XGBoost model: {model_file}")
    m = xgb.Booster()
    m.load_model(model_file)
    return m

def load_rf(model_dir):
    extract_dir = extract_tar(model_dir)
    candidates = glob.glob(os.path.join(extract_dir, "*.joblib"))
    if not candidates:
        raise FileNotFoundError(f"No .joblib file found in {extract_dir}")
    model_file = candidates[0]
    print(f"  Loading RF model: {model_file}")
    return joblib.load(model_file)

if __name__ == "__main__":
    try:
        # ── Diagnostics ───────────────────────────────────────
        print("=== EVALUATE MODELS ===")
        list_dir("/opt/ml/processing/model/xgboost")
        list_dir("/opt/ml/processing/model/rf")
        list_dir("/opt/ml/processing/test")

        # ── Load test data ─────────────────────────────────────
        test_files = glob.glob("/opt/ml/processing/test/*.csv")
        if not test_files:
            raise FileNotFoundError("No test CSV found in /opt/ml/processing/test/")
        print(f"  Test file: {test_files[0]}")
        test_df = pd.read_csv(test_files[0], header=None)
        print(f"  Test shape: {test_df.shape}")

        X_te = test_df.iloc[:, 1:].values.astype(float)
        y_te = test_df.iloc[:, 0].values.astype(float)

        results = {}

        # ── XGBoost evaluation ─────────────────────────────────
        try:
            print("\\nLoading XGBoost model ...")
            xgb_model = load_xgb("/opt/ml/processing/model/xgboost")
            dtest     = xgb.DMatrix(X_te)
            xgb_prob  = xgb_model.predict(dtest)
            xgb_pred  = (xgb_prob >= 0.5).astype(int)
            results["xgboost"] = {
                "accuracy":  round(float(accuracy_score(y_te, xgb_pred)),  4),
                "auc":       round(float(roc_auc_score(y_te, xgb_prob)),   4),
                "precision": round(float(precision_score(y_te, xgb_pred)), 4),
                "recall":    round(float(recall_score(y_te, xgb_pred)),    4),
                "f1":        round(float(f1_score(y_te, xgb_pred)),        4),
            }
            print(f"  XGBoost metrics: {results['xgboost']}")
        except Exception as e:
            print(f"  XGBoost eval failed: {e}")
            traceback.print_exc()
            results["xgboost"] = {"error": str(e), "accuracy": 0.0}

        # ── Random Forest evaluation ───────────────────────────
        try:
            print("\\nLoading Random Forest model ...")
            rf_model = load_rf("/opt/ml/processing/model/rf")
            rf_pred  = rf_model.predict(X_te)
            rf_prob  = rf_model.predict_proba(X_te)[:, 1]
            results["random_forest"] = {
                "accuracy":  round(float(accuracy_score(y_te, rf_pred)),  4),
                "auc":       round(float(roc_auc_score(y_te, rf_prob)),   4),
                "precision": round(float(precision_score(y_te, rf_pred)), 4),
                "recall":    round(float(recall_score(y_te, rf_pred)),    4),
                "f1":        round(float(f1_score(y_te, rf_pred)),        4),
            }
            print(f"  RF metrics: {results['random_forest']}")
        except Exception as e:
            print(f"  RF eval failed: {e}")
            traceback.print_exc()
            results["random_forest"] = {"error": str(e), "accuracy": 0.0}

        # ── Best model ─────────────────────────────────────────
        best = max(results, key=lambda k: results[k].get("accuracy", 0.0))
        results["best_model"] = best
        print(f"\\nBest model: {best}")
        print(json.dumps(results, indent=2))

        # ── Write evaluation report ────────────────────────────
        os.makedirs("/opt/ml/processing/evaluation", exist_ok=True)
        out_path = "/opt/ml/processing/evaluation/evaluation.json"
        with open(out_path, "w") as f:
            json.dump(results, f, indent=2)
        print(f"\\nEvaluation report written to {out_path}")

    except Exception as e:
        print("=" * 60)
        print(f"EVALUATION FAILED: {e}")
        traceback.print_exc()
        print("=" * 60)
        raise
"""

def save_training_scripts():
    scripts = {
        "xgb_train.py":   XGB_TRAIN_SCRIPT,
        "rf_train.py":    RF_TRAIN_SCRIPT,
        "evaluate.py":    EVALUATE_SCRIPT,
    }
    for fname, content in scripts.items():
        with open(fname, "w") as f:
            f.write(content)
        print(f"Script saved → {fname}")


# ─────────────────────────────────────────────────────────
# 6.  MODEL COMPARISON  (standalone, outside pipeline)
# ─────────────────────────────────────────────────────────

def compare_models_local(train_csv: str, test_csv: str) -> dict:
    """
    Quick local comparison of XGBoost vs RF vs Logistic Regression.
    Useful for rapid experimentation before launching full pipeline.
    """
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
    from sklearn.model_selection import cross_val_score
    import xgboost as xgb

    train = pd.read_csv(train_csv, header=None, encoding="latin-1")
    test  = pd.read_csv(test_csv,  header=None, encoding="latin-1")
    X_tr, y_tr = train.iloc[:, 1:].values, train.iloc[:, 0].values
    X_te, y_te = test.iloc[:, 1:].values,  test.iloc[:, 0].values

    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Random Forest":       RandomForestClassifier(n_estimators=200, random_state=42),
        "Gradient Boosting":   GradientBoostingClassifier(n_estimators=100, random_state=42),
        "XGBoost":             xgb.XGBClassifier(use_label_encoder=False,
                                                  eval_metric="logloss",
                                                  random_state=42),
    }

    results = {}
    print(f"\n{'Model':<25} {'Accuracy':>10} {'AUC':>10} {'F1':>10} {'CV-AUC (5-fold)':>18}")
    print("─" * 80)

    for name, m in models.items():
        m.fit(X_tr, y_tr)
        preds = m.predict(X_te)
        proba = m.predict_proba(X_te)[:, 1]
        cv_auc = cross_val_score(m, X_tr, y_tr, cv=5, scoring="roc_auc").mean()

        acc = accuracy_score(y_te, preds)
        auc = roc_auc_score(y_te, proba)
        f1  = f1_score(y_te, preds)

        results[name] = {"accuracy": acc, "auc": auc, "f1": f1, "cv_auc": cv_auc}
        print(f"{name:<25} {acc:>10.4f} {auc:>10.4f} {f1:>10.4f} {cv_auc:>18.4f}")

    best = max(results, key=lambda k: results[k]["auc"])
    print(f"\n🏆 Best model by AUC: {best} ({results[best]['auc']:.4f})")
    return results


# ─────────────────────────────────────────────────────────
# 7.  HYPERPARAMETER TUNING  (Automatic Model Tuning)
# ─────────────────────────────────────────────────────────

def run_hyperparameter_tuning(train_s3_uri: str) -> HyperparameterTuner:
    """
    SageMaker Automatic Model Tuning for XGBoost (built-in algorithm).
    Searches across eta, max_depth, subsample for best AUC.
    """
    xgb_image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost", region=region, version="1.5-1"
    )
    xgb_estimator = Estimator(
        image_uri=xgb_image_uri,
        instance_type="ml.m5.xlarge",
        instance_count=1,
        role=role,
        output_path=f"{OUTPUT_PREFIX}/tuning",
        sagemaker_session=sagemaker_session,
        hyperparameters={"num_round": 100, "objective": "binary:logistic", "eval_metric": "auc"}
    )

    tuner = HyperparameterTuner(
        estimator=xgb_estimator,
        objective_metric_name="validation:auc",
        hyperparameter_ranges={
            "eta":       ContinuousParameter(0.01, 0.3),
            "max_depth": IntegerParameter(3, 10),
            "subsample": ContinuousParameter(0.5, 1.0),
        },
        max_jobs=15,
        max_parallel_jobs=3,
        strategy="Bayesian"
    )

    tuner.fit(
        inputs={
            "train":      TrainingInput(train_s3_uri, content_type="text/csv"),
            "validation": TrainingInput(train_s3_uri, content_type="text/csv"),
        },
        wait=False
    )
    print(f"Tuning job started: {tuner.latest_tuning_job.name}")
    return tuner


# ─────────────────────────────────────────────────────────
# 8.  MODEL DEPLOYMENT
# ─────────────────────────────────────────────────────────

INFERENCE_SCRIPT = """
import os, io, json, tarfile
import numpy as np
import xgboost as xgb

def model_fn(model_dir):
    model = xgb.Booster()
    model.load_model(os.path.join(model_dir, "xgboost-model"))
    return model

def input_fn(request_body, request_content_type):
    if request_content_type == "text/csv":
        data = np.array([float(x) for x in request_body.strip().split(",")])
        return xgb.DMatrix(data.reshape(1, -1))
    elif request_content_type == "application/json":
        d = json.loads(request_body)
        data = np.array(d["features"])
        return xgb.DMatrix(data.reshape(1, -1))
    raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    probability = float(model.predict(input_data)[0])
    return probability

def output_fn(prediction, accept):
    result = {
        "probability": round(prediction, 4),
        "prediction":  int(prediction >= 0.5),
        "label":       "liver_disease" if prediction >= 0.5 else "no_disease"
    }
    return json.dumps(result), "application/json"
"""

def deploy_model(model_data_s3: str, endpoint_name: str = "liver-disease-endpoint") -> sagemaker.predictor.Predictor:
    """
    Deploy best XGBoost model to a real-time SageMaker endpoint
    with data capture enabled for monitoring.
    """
    with open("inference.py", "w") as f:
        f.write(INFERENCE_SCRIPT)

    # Data Capture Config (required for Model Monitor)
    data_capture_config = DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{BUCKET}/{PREFIX}/data-capture",
        capture_options=["REQUEST", "RESPONSE"]
    )

    xgb_image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost", region=region, version="1.5-1"
    )
    model = sagemaker.Model(
        image_uri=xgb_image_uri,
        model_data=model_data_s3,
        role=role,
        sagemaker_session=sagemaker_session,
        env={"SAGEMAKER_CONTAINER_LOG_LEVEL": "20"}
    )

    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name=endpoint_name,
        data_capture_config=data_capture_config
    )

    print(f"✅ Endpoint deployed: {endpoint_name}")
    return predictor


def test_endpoint(predictor, sample_features: list):
    """Send a test prediction to the deployed endpoint."""
    import csv, io
    payload = ",".join(map(str, sample_features))
    response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
    print(f"Prediction: {response}")
    return response


# ─────────────────────────────────────────────────────────
# 9.  MODEL MONITORING
# ─────────────────────────────────────────────────────────

def setup_model_monitoring(endpoint_name: str, baseline_s3_uri: str):
    """
    Sets up two monitoring schedules:
      1. Data Quality Monitor  – detects data drift / schema violations
      2. Model Quality Monitor – tracks accuracy / AUC over time
    """

    # ── Data Quality Monitor ──────────────────────────────
    data_monitor = DefaultModelMonitor(
        role=role,
        instance_count=1,
        instance_type="ml.m5.xlarge",
        volume_size_in_gb=20,
        max_runtime_in_seconds=3600,
        sagemaker_session=sagemaker_session
    )

    # Suggest baseline constraints & statistics from training data
    data_monitor.suggest_baseline(
        baseline_dataset=baseline_s3_uri,
        dataset_format=DatasetFormat.csv(header=True),
        output_s3_uri=f"s3://{BUCKET}/{PREFIX}/monitoring/baseline",
        wait=True
    )
    print("Baseline statistics computed.")

    # Schedule hourly monitoring
    data_monitor.create_monitoring_schedule(
        monitor_schedule_name="liver-data-quality-monitor",
        endpoint_input=endpoint_name,
        output_s3_uri=f"s3://{BUCKET}/{PREFIX}/monitoring/data-quality",
        statistics=data_monitor.baseline_statistics(),
        constraints=data_monitor.suggested_constraints(),
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True
    )
    print("Data Quality Monitor schedule created (hourly).")

    # ── Model Quality Monitor ─────────────────────────────
    model_quality_monitor = ModelQualityMonitor(
        role=role,
        instance_count=1,
        instance_type="ml.m5.xlarge",
        volume_size_in_gb=20,
        max_runtime_in_seconds=1800,
        sagemaker_session=sagemaker_session
    )

    model_quality_monitor.create_monitoring_schedule(
        monitor_schedule_name="liver-model-quality-monitor",
        endpoint_input=sagemaker.model_monitor.EndpointInput(
            endpoint_name=endpoint_name,
            destination="/opt/ml/processing/input_data",
            start_time_offset="-PT1H",
            end_time_offset="-PT0H",
            probability_threshold_attribute=0.5,
        ),
        ground_truth_input=f"s3://{BUCKET}/{PREFIX}/ground-truth/",
        problem_type="BinaryClassification",
        output_s3_uri=f"s3://{BUCKET}/{PREFIX}/monitoring/model-quality",
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True
    )
    print("Model Quality Monitor schedule created (hourly).")

    return data_monitor, model_quality_monitor


def check_monitoring_violations(schedule_name: str):
    """List recent monitoring violations."""
    executions = sm_client.list_monitoring_executions(
        MonitoringScheduleName=schedule_name,
        SortBy="ScheduledTime",
        SortOrder="Descending",
        MaxResults=5
    )["MonitoringExecutionSummaries"]

    print(f"\nLatest executions for '{schedule_name}':")
    for ex in executions:
        print(f"  {ex['ScheduledTime']} → {ex['MonitoringExecutionStatus']}")
        if ex.get("FailureReason"):
            print(f"    ⚠ {ex['FailureReason']}")

    return executions


# ─────────────────────────────────────────────────────────
# 10.  CI/CD PIPELINE  (CodePipeline + SageMaker)
# ─────────────────────────────────────────────────────────

BUILDSPEC_YAML = """
version: 0.2
phases:
  install:
    runtime-versions:
      python: 3.9
    commands:
      - pip install sagemaker boto3 pandas scikit-learn xgboost

  pre_build:
    commands:
      - echo "Running unit tests ..."
      - python -m pytest tests/ -v --tb=short

  build:
    commands:
      - echo "Triggering SageMaker Pipeline ..."
      - python cicd_trigger.py

  post_build:
    commands:
      - echo "Build complete. Checking pipeline status ..."
      - python cicd_check_status.py

artifacts:
  files:
    - "**/*"
"""

CICD_TRIGGER_SCRIPT = """
import boto3, json
sm = boto3.client("sagemaker")

response = sm.start_pipeline_execution(
    PipelineName="LiverDiseasePipeline",
    PipelineExecutionDisplayName="cicd-triggered",
    PipelineParameters=[
        {"Name": "ModelApprovalStatus",  "Value": "Approved"},
        {"Name": "AccuracyThreshold",    "Value": "0.75"},
    ]
)
execution_arn = response["PipelineExecutionArn"]
print(f"Pipeline execution started: {execution_arn}")

# Write ARN for next step
with open("execution_arn.txt", "w") as f:
    f.write(execution_arn)
"""

CICD_STATUS_SCRIPT = """
import boto3, time

sm = boto3.client("sagemaker")
with open("execution_arn.txt") as f:
    arn = f.read().strip()

for _ in range(60):        # poll for up to 60 minutes
    r      = sm.describe_pipeline_execution(PipelineExecutionArn=arn)
    status = r["PipelineExecutionStatus"]
    print(f"Status: {status}")
    if status in ("Succeeded", "Failed", "Stopped"):
        break
    time.sleep(60)

if status != "Succeeded":
    raise SystemExit(f"Pipeline did not succeed: {status}")
print("Pipeline succeeded! Model ready for deployment.")
"""

UNIT_TEST_SCRIPT = """
import pytest, pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler

def test_preprocessing_no_nulls():
    df = pd.read_csv("Liver_Patient_Dataset__LPD__train.csv", encoding="latin-1")
    df.dropna(inplace=True)
    assert df.isnull().sum().sum() == 0

def test_target_binary():
    df = pd.read_csv("Liver_Patient_Dataset__LPD__train.csv", encoding="latin-1")
    df.dropna(inplace=True)
    df["Result"] = (df["Result"] == 1).astype(int)
    assert set(df["Result"].unique()).issubset({0, 1})

def test_feature_count():
    df = pd.read_csv("Liver_Patient_Dataset__LPD__train.csv", encoding="latin-1")
    df.dropna(inplace=True)
    # 11 original + 3 engineered - 1 target = 13 features
    assert df.shape[1] >= 11

def test_scaler_mean_zero():
    data = np.random.randn(100, 5)
    scaler = StandardScaler()
    scaled = scaler.fit_transform(data)
    assert abs(scaled.mean()) < 0.1
"""

def setup_cicd_files():
    """Write all CI/CD related files to disk."""
    files = {
        "buildspec.yml":         BUILDSPEC_YAML,
        "cicd_trigger.py":       CICD_TRIGGER_SCRIPT,
        "cicd_check_status.py":  CICD_STATUS_SCRIPT,
        "tests/test_pipeline.py": UNIT_TEST_SCRIPT,
    }
    import os
    os.makedirs("tests", exist_ok=True)
    for path, content in files.items():
        with open(path, "w") as f:
            f.write(content)
    print("CI/CD files created.")


def create_codepipeline(github_repo: str, github_branch: str = "main"):
    """
    Creates an AWS CodePipeline that:
      Source  → GitHub (on push to main)
      Build   → CodeBuild (runs buildspec.yml)
      Deploy  → triggers SageMaker Pipeline
    """
    cp_client = boto_session.client("codepipeline")

    pipeline_definition = {
        "name":    "LiverDiseaseMLOps",
        "roleArn": role,
        "artifactStore": {
            "type":     "S3",
            "location": BUCKET
        },
        "stages": [
            {
                "name": "Source",
                "actions": [{
                    "name":       "GitHub_Source",
                    "actionTypeId": {
                        "category": "Source",
                        "owner":    "ThirdParty",
                        "provider": "GitHub",
                        "version":  "1"
                    },
                    "configuration": {
                        "Owner":      github_repo.split("/")[0],
                        "Repo":       github_repo.split("/")[1],
                        "Branch":     github_branch,
                        "OAuthToken": "{{resolve:secretsmanager:github-token}}"
                    },
                    "outputArtifacts": [{"name": "SourceOutput"}]
                }]
            },
            {
                "name": "Build",
                "actions": [{
                    "name": "CodeBuild",
                    "actionTypeId": {
                        "category": "Build",
                        "owner":    "AWS",
                        "provider": "CodeBuild",
                        "version":  "1"
                    },
                    "configuration": {
                        "ProjectName": "LiverDiseaseMLBuild"
                    },
                    "inputArtifacts":  [{"name": "SourceOutput"}],
                    "outputArtifacts": [{"name": "BuildOutput"}]
                }]
            }
        ]
    }

    try:
        cp_client.create_pipeline(pipeline=pipeline_definition)
        print("CodePipeline 'LiverDiseaseMLOps' created.")
    except cp_client.exceptions.PipelineNameInUseException:
        cp_client.update_pipeline(pipeline=pipeline_definition)
        print("CodePipeline 'LiverDiseaseMLOps' updated.")


# ─────────────────────────────────────────────────────────
# 11.  CLOUDWATCH INFRASTRUCTURE MONITORING DASHBOARD
# ─────────────────────────────────────────────────────────

def create_cloudwatch_dashboard(endpoint_name: str = "liver-disease-endpoint"):
    """
    Creates a CloudWatch dashboard with widgets covering:
      - Endpoint invocation metrics (latency, throughput, errors)
      - Instance-level CPU / Memory utilisation
      - Model Monitor violation counts
      - Training job metrics (XGBoost AUC over rounds)
      - SageMaker Pipeline execution status
    """
    cw = boto_session.client("cloudwatch")
    dashboard_name = "LiverDisease-MLOps-Dashboard"

    # Helper: metric widget definition
    def metric_widget(title, metrics, period=300, stat="Average",
                      width=12, height=6, view="timeSeries"):
        return {
            "type": "metric",
            "properties": {
                "title":   title,
                "view":    view,
                "stacked": False,
                "stat":    stat,
                "period":  period,
                "metrics": metrics,
                "width":   width,
                "height":  height,
            }
        }

    # Helper: alarm widget
    def alarm_widget(title, alarm_arns, width=12, height=3):
        return {
            "type": "alarm",
            "properties": {
                "title":  title,
                "alarms": alarm_arns,
                "width":  width,
                "height": height,
            }
        }

    widgets = [
        # ── Row 1: Endpoint Invocation Health ────────────
        metric_widget(
            title="Endpoint – Invocations per Minute",
            metrics=[
                ["AWS/SageMaker", "Invocations",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
                ["AWS/SageMaker", "InvocationsPerInstance",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
            ],
            stat="Sum"
        ),
        metric_widget(
            title="Endpoint – Model Latency (ms)",
            metrics=[
                ["AWS/SageMaker", "ModelLatency",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
                ["AWS/SageMaker", "OverheadLatency",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
            ]
        ),
        # ── Row 2: Error Rates ────────────────────────────
        metric_widget(
            title="Endpoint – 4xx / 5xx Errors",
            metrics=[
                ["AWS/SageMaker", "Invocation4XXErrors",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
                ["AWS/SageMaker", "Invocation5XXErrors",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
            ],
            stat="Sum"
        ),
        # ── Row 3: Instance Resource Utilisation ─────────
        metric_widget(
            title="Endpoint Instance – CPU & Memory Utilisation (%)",
            metrics=[
                ["AWS/SageMaker", "CPUUtilization",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
                ["AWS/SageMaker", "MemoryUtilization",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
            ]
        ),
        metric_widget(
            title="Endpoint Instance – Disk Utilisation (%)",
            metrics=[
                ["AWS/SageMaker", "DiskUtilization",
                 "EndpointName", endpoint_name, "VariantName", "AllTraffic"],
            ]
        ),
        # ── Row 4: Model Monitor Violations ───────────────
        metric_widget(
            title="Data Quality Monitor – Violation Count",
            metrics=[
                ["/aws/sagemaker/Endpoints/data-metrics", "feature_baseline_drift_violations",
                 "MonitoringSchedule", "liver-data-quality-monitor"],
            ],
            stat="Sum"
        ),
        metric_widget(
            title="Model Quality Monitor – Violation Count",
            metrics=[
                ["/aws/sagemaker/Endpoints/data-metrics", "metric_violations",
                 "MonitoringSchedule", "liver-model-quality-monitor"],
            ],
            stat="Sum"
        ),
        # ── Row 5: Feature Store ──────────────────────────
        metric_widget(
            title="Feature Store – Put / Get Record Latency (ms)",
            metrics=[
                ["AWS/SageMaker", "PutRecord.Latency",
                 "FeatureGroupName", FEATURE_GROUP_NAME],
                ["AWS/SageMaker", "GetRecord.Latency",
                 "FeatureGroupName", FEATURE_GROUP_NAME],
            ]
        ),
    ]

    body = json.dumps({"widgets": widgets})
    dashboard_created = False
    try:
        cw.put_dashboard(DashboardName=dashboard_name, DashboardBody=body)
        dashboard_created = True
        print(f"✅ CloudWatch dashboard created: '{dashboard_name}'")
        print(f"   → https://console.aws.amazon.com/cloudwatch/home#dashboards:name={dashboard_name}")
    except Exception as e:
        print(f"⚠  Could not create CloudWatch dashboard (IAM permission missing: cloudwatch:PutDashboard)")
        print(f"   Error: {e}")
        print()
        print("   ── To create the dashboard manually ─────────────────────────")
        print("   1. Open AWS Console → CloudWatch → Dashboards → Create dashboard")
        print(f"   2. Name it: {dashboard_name}")
        print("   3. Add widgets for these metrics:")
        print(f"      • AWS/SageMaker → EndpointName={endpoint_name} → Invocations, ModelLatency")
        print(f"      • AWS/SageMaker → EndpointName={endpoint_name} → CPUUtilization, MemoryUtilization")
        print(f"      • AWS/SageMaker → EndpointName={endpoint_name} → Invocation4XXErrors, Invocation5XXErrors")
        print("   ─────────────────────────────────────────────────────────────")

    # ── CloudWatch Alarms ─────────────────────────────────
    alarms = [
        {
            "AlarmName":          "LiverDisease-HighLatency",
            "AlarmDescription":   "Model latency exceeds 500ms",
            "MetricName":         "ModelLatency",
            "Namespace":          "AWS/SageMaker",
            "Dimensions":         [{"Name": "EndpointName", "Value": endpoint_name},
                                   {"Name": "VariantName",  "Value": "AllTraffic"}],
            "Statistic":          "Average",
            "Period":             60,
            "EvaluationPeriods":  3,
            "Threshold":          500000,   # microseconds
            "ComparisonOperator": "GreaterThanThreshold",
            "TreatMissingData":   "notBreaching",
        },
        {
            "AlarmName":          "LiverDisease-5xxErrors",
            "AlarmDescription":   "Endpoint returning 5xx errors",
            "MetricName":         "Invocation5XXErrors",
            "Namespace":          "AWS/SageMaker",
            "Dimensions":         [{"Name": "EndpointName", "Value": endpoint_name},
                                   {"Name": "VariantName",  "Value": "AllTraffic"}],
            "Statistic":          "Sum",
            "Period":             60,
            "EvaluationPeriods":  2,
            "Threshold":          1,
            "ComparisonOperator": "GreaterThanOrEqualToThreshold",
            "TreatMissingData":   "notBreaching",
        },
        {
            "AlarmName":          "LiverDisease-HighCPU",
            "AlarmDescription":   "Endpoint CPU > 80%",
            "MetricName":         "CPUUtilization",
            "Namespace":          "AWS/SageMaker",
            "Dimensions":         [{"Name": "EndpointName", "Value": endpoint_name},
                                   {"Name": "VariantName",  "Value": "AllTraffic"}],
            "Statistic":          "Average",
            "Period":             300,
            "EvaluationPeriods":  2,
            "Threshold":          80,
            "ComparisonOperator": "GreaterThanThreshold",
            "TreatMissingData":   "notBreaching",
        },
    ]

    for alarm in alarms:
        try:
            cw.put_metric_alarm(**alarm)
            print(f"  Alarm set → {alarm['AlarmName']}")
        except Exception as e:
            print(f"  ⚠  Alarm '{alarm['AlarmName']}' skipped (check IAM): {e}")

    return dashboard_name


# ─────────────────────────────────────────────────────────
# 12.  CI/CD DAG  – SUCCESS & FAILED STATE HANDLER
# ─────────────────────────────────────────────────────────

def visualise_pipeline_dag(execution_arn: str):
    """
    Prints a text-based DAG of the pipeline execution showing
    each step's status.  Clearly shows SUCCESS and FAILED states
    with actionable failure messages.
    """
    steps = sm_client.list_pipeline_execution_steps(
        PipelineExecutionArn=execution_arn,
        SortOrder="Ascending"
    )["PipelineExecutionSteps"]

    STATUS_ICON = {
        "Succeeded":  "✅",
        "Failed":     "❌",
        "Executing":  "🔄",
        "Stopped":    "⏹️",
        "Starting":   "⏳",
        "Skipped":    "⏭️",
    }

    DAG_SHAPE = [
        ("LiverDiseasePreprocessing", None),
        ("TrainXGBoost",              "LiverDiseasePreprocessing"),
        ("TrainRandomForest",         "LiverDiseasePreprocessing"),
        ("EvaluateModels",            "TrainXGBoost + TrainRandomForest"),
        ("CheckAccuracy",             "EvaluateModels"),
        ("RegisterLiverDiseaseModel", "CheckAccuracy"),
    ]

    step_map = {s["StepName"]: s for s in steps}

    print("\n" + "═" * 62)
    print("  CI/CD PIPELINE DAG")
    print("═" * 62)

    any_failed = False
    for step_name, parent in DAG_SHAPE:
        info   = step_map.get(step_name, {})
        status = info.get("StepStatus", "NotStarted")
        icon   = STATUS_ICON.get(status, "⬜")
        indent = "      └─► " if parent else "  "

        print(f"{indent}{icon}  {step_name:<38} [{status}]")

        # Show failure reason inline
        if status == "Failed":
            any_failed = True
            reason = info.get("FailureReason", "No reason provided")
            print(f"           ⚠  Failure: {reason}")

        # Show condition branch outcome
        if step_name == "CheckAccuracy" and status == "Succeeded":
            metadata = info.get("Metadata", {})
            outcome  = metadata.get("Condition", {}).get("Outcome", "unknown")
            print(f"           ↳  Condition outcome: {outcome.upper()}")
            if outcome == "false":
                print("           ↳  Model NOT registered (accuracy below threshold)")

    print("═" * 62)

    # ── Overall result ────────────────────────────────────
    execution_info = sm_client.describe_pipeline_execution(
        PipelineExecutionArn=execution_arn
    )
    overall = execution_info["PipelineExecutionStatus"]
    print(f"\n  Overall Status : {STATUS_ICON.get(overall, '?')}  {overall}")

    if overall == "Failed" or any_failed:
        print("\n  ── FAILURE REPORT ──────────────────────────────────")
        for step_name, _ in DAG_SHAPE:
            info = step_map.get(step_name, {})
            if info.get("StepStatus") == "Failed":
                print(f"  Step  : {step_name}")
                print(f"  Reason: {info.get('FailureReason', 'N/A')}")
                print(f"  ARN   : {info.get('Metadata', {})}")
                print()
        print("  ── REMEDIATION HINTS ───────────────────────────────")
        print("  1. Check CloudWatch Logs for the failed Processing/Training job")
        print("  2. Verify IAM role has s3:PutObject on the output bucket")
        print("  3. If CheckAccuracy failed: lower AccuracyThreshold parameter")
        print("  4. Re-run:  execution = pipeline.start(...)")
    else:
        print("\n  ── SUCCESS SUMMARY ─────────────────────────────────")
        print("  All steps completed. Model registered in Model Registry.")
        print("  Next: approve model → deploy endpoint → enable monitoring.")

    print("═" * 62 + "\n")
    return overall


def monitor_pipeline_execution(pipeline_name: str = "LiverDiseasePipeline",
                                poll_seconds: int = 30):
    """
    Starts the pipeline and live-polls it, printing DAG updates
    until it reaches a terminal state (Succeeded / Failed / Stopped).
    Handles both SUCCESS and FAILED outcomes with full reporting.
    """
    # Start execution
    response = sm_client.start_pipeline_execution(
        PipelineName=pipeline_name,
        PipelineParameters=[
            {"Name": "ModelApprovalStatus", "Value": "PendingManualApproval"},
            {"Name": "AccuracyThreshold",   "Value": "0.75"},
        ]
    )
    arn = response["PipelineExecutionArn"]
    print(f"Pipeline execution started:\n  {arn}\n")

    terminal = {"Succeeded", "Failed", "Stopped"}
    while True:
        status = sm_client.describe_pipeline_execution(
            PipelineExecutionArn=arn
        )["PipelineExecutionStatus"]

        visualise_pipeline_dag(arn)

        if status in terminal:
            break

        print(f"  Polling again in {poll_seconds}s …\n")
        time.sleep(poll_seconds)

    # Final SNS notification (optional – requires an SNS topic)
    _send_pipeline_notification(arn, status)
    return arn, status


def _send_pipeline_notification(execution_arn: str, status: str):
    """SNS requires sns:CreateTopic + sns:Publish IAM permissions not available
    on this SageMaker role. No-op here — silently skipped.
    To enable in production: add SNS permissions to the IAM role, then
    call sns.create_topic() + sns.publish() here.
    """
    pass  # no-op — IAM permissions not available


# ─────────────────────────────────────────────────────────
# 13.  BATCH INFERENCE  +  ENDPOINT INVOCATION EXAMPLES
# ─────────────────────────────────────────────────────────

def run_batch_transform(model_name: str, test_s3_uri: str):
    """
    SageMaker Batch Transform job — runs inference on the full
    test set in S3 and writes predictions back to S3.
    No endpoint needed; ideal for offline/bulk scoring.
    """
    transformer = sagemaker.transformer.Transformer(
        model_name=model_name,
        instance_count=1,
        instance_type="ml.m5.xlarge",
        output_path=f"s3://{BUCKET}/{PREFIX}/batch-output",
        base_transform_job_name="liver-disease-batch",
        sagemaker_session=sagemaker_session,
        accept="text/csv",
        assemble_with="Line",
        strategy="MultiRecord",
    )

    # data_type: use S3Prefix for directories, ManifestFile for single files
    data_type = "S3Prefix" if not test_s3_uri.endswith(".csv") else "S3Prefix"

    transformer.transform(
        data=test_s3_uri,
        data_type=data_type,
        content_type="text/csv",
        split_type="Line",
        wait=True,
        logs=False   # suppress verbose nginx/gunicorn logs
    )

    output_uri = transformer.output_path
    print(f"\n✅ Batch transform complete.")
    print(f"   Output → {output_uri}")

    # ── Download & display sample predictions ─────────────
    s3 = boto_session.resource("s3")
    bucket_obj = s3.Bucket(BUCKET)
    out_prefix = f"{PREFIX}/batch-output"

    print("\n── Sample Batch Predictions ────────────────────────")
    print(f"{'Row':<6} {'Raw Score':>12} {'Prediction':>12} {'Label':>18}")
    print("─" * 55)

    for i, obj in enumerate(bucket_obj.objects.filter(Prefix=out_prefix)):
        content = obj.get()["Body"].read().decode("utf-8")
        for j, line in enumerate(content.strip().split("\n")[:10]):
            score = float(line.strip())
            pred  = int(score >= 0.5)
            label = "liver_disease" if pred == 1 else "no_disease"
            print(f"{j:<6} {score:>12.4f} {pred:>12} {label:>18}")
        break   # just show first output file

    return output_uri


def invoke_endpoint_examples(endpoint_name: str = "liver-disease-endpoint"):
    """
    Demonstrates three ways to call the deployed endpoint:
      1. Single record (CSV)
      2. Single record (JSON)
      3. Batch of records via loop
    Uses real feature values from the dataset.
    """
    runtime = boto_session.client("sagemaker-runtime")

    # Sample patients from the dataset (post-preprocessing, normalised approx values)
    sample_patients = [
        # Age  Gender  TotalBili  DirBili  AlkPhos  SGPT   SGOT  TotProt  Albumin  AGRatio  BiliRatio  EnzRatio  AgeGrp
        [65,    1,     0.7,       0.1,     187,     16,    18,   6.8,     3.3,     0.90,    0.14,      0.89,     2],  # likely disease
        [32,    0,     0.9,       0.2,     200,     20,    22,   7.0,     3.5,     1.10,    0.22,      0.91,     1],  # likely no disease
        [55,    1,     3.5,       1.2,     450,     120,   95,   5.5,     2.8,     0.70,    0.34,      1.26,     2],  # high risk
    ]

    labels = ["Patient A (65M)", "Patient B (32F)", "Patient C (55M – high risk)"]

    print("\n" + "═" * 65)
    print("  ENDPOINT INVOCATION EXAMPLES")
    print("═" * 65)

    # ── Method 1: CSV payload ─────────────────────────────
    print("\n── Method 1: CSV Payload ───────────────────────────")
    for patient, label in zip(sample_patients, labels):
        payload = ",".join(map(str, patient))
        response = runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload
        )
        result = json.loads(response["Body"].read().decode("utf-8"))
        print(f"  {label:<28} → prob={result['probability']:.4f}  "
              f"pred={result['prediction']}  ({result['label']})")

    # ── Method 2: JSON payload ────────────────────────────
    print("\n── Method 2: JSON Payload ──────────────────────────")
    payload = json.dumps({"features": sample_patients[0]})
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=payload
    )
    result = json.loads(response["Body"].read().decode("utf-8"))
    print(f"  JSON response: {json.dumps(result, indent=4)}")

    # ── Method 3: Batch loop with latency tracking ────────
    print("\n── Method 3: Bulk Invocation (100 records) ─────────")
    latencies, predictions = [], []
    for _ in range(100):
        patient = sample_patients[_ % len(sample_patients)]
        payload = ",".join(map(str, patient))
        t0 = time.time()
        response = runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload
        )
        latency_ms = (time.time() - t0) * 1000
        result = json.loads(response["Body"].read().decode("utf-8"))
        latencies.append(latency_ms)
        predictions.append(result["prediction"])

    disease_pct = sum(predictions) / len(predictions) * 100
    print(f"  Records invokedé: 100")
    print(f"  Avg latency  : {np.mean(latencies):.1f} ms")
    print(f"  p95 latency  : {np.percentile(latencies, 95):.1f} ms")
    print(f"  Max latency  : {np.max(latencies):.1f} ms")
    print(f"  Disease rate : {disease_pct:.1f}%")
    print("═" * 65)

    return predictions, latencies


# ─────────────────────────────────────────────────────────
# 14.  MONITORING REPORT GENERATION
# ─────────────────────────────────────────────────────────

def generate_monitoring_report(endpoint_name: str = "liver-disease-endpoint"):
    """
    Pulls the latest monitoring execution results from S3,
    parses violations and statistics, and prints a full report.
    Covers both Data Quality and Model Quality monitors.
    """
    print("\n" + "═" * 65)
    print("  MODEL MONITORING REPORT")
    print(f"  Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")
    print("═" * 65)

    # ── Helper: fetch latest execution output from S3 ─────
    def get_latest_execution_uri(schedule_name: str) -> str | None:
        try:
            execs = sm_client.list_monitoring_executions(
                MonitoringScheduleName=schedule_name,
                SortBy="ScheduledTime",
                SortOrder="Descending",
                MaxResults=1
            )["MonitoringExecutionSummaries"]
            if not execs:
                return None
            return execs[0].get("ProcessingJobArn")
        except Exception:
            return None

    def read_s3_json(s3_uri: str) -> dict:
        """Read a JSON file from S3 URI."""
        parts  = s3_uri.replace("s3://", "").split("/", 1)
        bucket = parts[0]
        key    = parts[1]
        obj    = s3_client.get_object(Bucket=bucket, Key=key)
        return json.loads(obj["Body"].read().decode("utf-8"))

    def list_s3_files(prefix_uri: str) -> list:
        parts  = prefix_uri.replace("s3://", "").split("/", 1)
        bucket = parts[0]
        prefix = parts[1]
        resp   = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
        return [f"s3://{bucket}/{o['Key']}" for o in resp.get("Contents", [])]

    # ── Section 1: Data Quality Report ────────────────────
    print("\n── DATA QUALITY MONITOR ────────────────────────────")
    dq_output_prefix = f"s3://{BUCKET}/{PREFIX}/monitoring/data-quality"
    dq_files = list_s3_files(dq_output_prefix)

    violations_file  = next((f for f in dq_files if "constraint_violations" in f), None)
    statistics_file  = next((f for f in dq_files if "statistics.json" in f), None)
    constraints_file = next((f for f in dq_files if "constraints.json" in f), None)

    if violations_file:
        violations = read_s3_json(violations_file)
        vlist      = violations.get("violations", [])
        print(f"\n  Total Violations : {len(vlist)}")

        if vlist:
            print(f"\n  {'Feature':<35} {'Constraint':<20} {'Description'}")
            print("  " + "─" * 75)
            for v in vlist:
                print(f"  {v.get('feature_name','?'):<35} "
                      f"{v.get('constraint_check_type','?'):<20} "
                      f"{v.get('description','?')}")
        else:
            print("  ✅ No data quality violations detected.")
    else:
        print("  ⚠  No violation report found yet (monitor may not have run).")

    if statistics_file:
        stats = read_s3_json(statistics_file)
        print(f"\n  ── Feature Statistics (latest execution) ──────────")
        print(f"  {'Feature':<35} {'Mean':>10} {'Std':>10} {'Missing%':>10}")
        print("  " + "─" * 70)
        for feat in stats.get("features", []):
            name    = feat.get("name", "?")
            num     = feat.get("numerical_statistics", {})
            mean    = num.get("mean",   "N/A")
            std     = num.get("std_dev","N/A")
            missing = feat.get("string_statistics", {}).get(
                "common_values", [{}])[0].get("value", "N/A")
            pct_missing = feat.get("num_missing_data_points", 0)
            total       = feat.get("num_data_points", 1)
            miss_pct    = f"{100 * pct_missing / total:.1f}%" if total else "N/A"
            mean_s  = f"{mean:.4f}" if isinstance(mean, float) else str(mean)
            std_s   = f"{std:.4f}"  if isinstance(std, float)  else str(std)
            print(f"  {name:<35} {mean_s:>10} {std_s:>10} {miss_pct:>10}")

    # ── Section 2: Model Quality Report ───────────────────
    print("\n── MODEL QUALITY MONITOR ───────────────────────────")
    mq_output_prefix = f"s3://{BUCKET}/{PREFIX}/monitoring/model-quality"
    mq_files = list_s3_files(mq_output_prefix)

    mq_violations = next((f for f in mq_files if "constraint_violations" in f), None)
    mq_stats      = next((f for f in mq_files if "statistics.json" in f), None)

    if mq_stats:
        stats  = read_s3_json(mq_stats)
        binary = stats.get("binary_classification_metrics", {})

        print(f"\n  {'Metric':<30} {'Value':>10} {'Threshold':>12} {'Status':>10}")
        print("  " + "─" * 65)

        metric_thresholds = {
            "accuracy":  0.75,
            "auc":       0.75,
            "precision": 0.70,
            "recall":    0.70,
            "f1":        0.70,
        }
        for metric, threshold in metric_thresholds.items():
            val    = binary.get(metric, {}).get("value", None)
            if val is None:
                print(f"  {metric:<30} {'N/A':>10} {threshold:>12.2f} {'?':>10}")
                continue
            status = "✅ OK" if val >= threshold else "❌ FAIL"
            print(f"  {metric:<30} {val:>10.4f} {threshold:>12.2f} {status:>10}")
    else:
        print("  ⚠  No model quality statistics found yet.")

    if mq_violations:
        mv = read_s3_json(mq_violations).get("violations", [])
        print(f"\n  Model Quality Violations: {len(mv)}")
        for v in mv:
            print(f"    ❌ {v.get('metric_name','?')} — {v.get('description','?')}")

    # ── Section 3: Endpoint Traffic Summary ───────────────
    print("\n── ENDPOINT TRAFFIC (last 1 hour via CloudWatch) ────")
    cw = boto_session.client("cloudwatch")
    now   = datetime.utcnow()
    start = now.replace(minute=0, second=0, microsecond=0)

    def get_cw_metric(metric_name, stat="Average"):
        resp = cw.get_metric_statistics(
            Namespace="AWS/SageMaker",
            MetricName=metric_name,
            Dimensions=[
                {"Name": "EndpointName", "Value": endpoint_name},
                {"Name": "VariantName",  "Value": "AllTraffic"},
            ],
            StartTime=start,
            EndTime=now,
            Period=3600,
            Statistics=[stat]
        )
        pts = resp.get("Datapoints", [])
        return pts[0][stat] if pts else None

    invocations = get_cw_metric("Invocations", "Sum")
    latency_avg = get_cw_metric("ModelLatency", "Average")
    errors_5xx  = get_cw_metric("Invocation5XXErrors", "Sum")
    cpu_avg     = get_cw_metric("CPUUtilization", "Average")

    print(f"  Invocations (1h)    : {invocations or 'N/A'}")
    print(f"  Avg Model Latency   : {f'{latency_avg/1000:.1f} ms' if latency_avg else 'N/A'}")
    print(f"  5xx Errors          : {int(errors_5xx) if errors_5xx else 0}")
    print(f"  Avg CPU Utilisation : {f'{cpu_avg:.1f}%' if cpu_avg else 'N/A'}")

    # ── Section 4: Model Registry Summary ─────────────────
    print("\n── MODEL REGISTRY ──────────────────────────────────")
    try:
        packages = sm_client.list_model_packages(
            ModelPackageGroupName="LiverDiseaseModelGroup",
            SortBy="CreationTime",
            SortOrder="Descending",
            MaxResults=5
        )["ModelPackageSummaryList"]

        print(f"  {'Version':<10} {'Status':<25} {'Created':<25} {'ARN'}")
        print("  " + "─" * 90)
        for pkg in packages:
            ver     = pkg.get("ModelPackageVersion", "?")
            status  = pkg.get("ModelApprovalStatus", "?")
            created = pkg["CreationTime"].strftime("%Y-%m-%d %H:%M")
            arn     = pkg["ModelPackageArn"]
            icon    = "✅" if status == "Approved" else ("❌" if status == "Rejected" else "⏳")
            print(f"  {ver:<10} {icon} {status:<22} {created:<25} {arn}")
    except Exception as e:
        print(f"  ⚠  Could not fetch model registry: {e}")

    print("\n" + "═" * 65)
    print("  END OF MONITORING REPORT")
    print("═" * 65 + "\n")


# ─────────────────────────────────────────────────────────
# 15.  EXTRACT PIPELINE ARTIFACTS  (auto-parse execution output)
# ─────────────────────────────────────────────────────────

def extract_pipeline_artifacts(execution_arn: str) -> dict:
    """
    Automatically extracts all ARNs, S3 URIs and names produced
    by a completed pipeline execution. No manual copy-pasting needed.

    Returns a dict with keys:
        model_data_s3       – S3 URI of the XGBoost model.tar.gz
        model_package_arn   – ARN of the registered Model Package
        model_package_name  – same ARN (used as name in update_model_package)
        training_job_name   – name of the XGBoost training job
        baseline_s3_uri     – S3 URI of the monitoring baseline CSV
        test_s3_uri         – S3 URI of the test dataset
        evaluation_results  – parsed evaluation.json dict
    """
    print("\n" + "═" * 62)
    print("  EXTRACTING PIPELINE ARTIFACTS")
    print("═" * 62)

    steps = sm_client.list_pipeline_execution_steps(
        PipelineExecutionArn=execution_arn,
        SortOrder="Ascending"
    )["PipelineExecutionSteps"]

    step_map = {s["StepName"]: s for s in steps}
    artifacts = {}

    # ── XGBoost Training Job → model_data_s3 ─────────────
    xgb_step = step_map.get("TrainXGBoost", {})
    xgb_job_arn = xgb_step.get("Metadata", {}).get("TrainingJob", {}).get("Arn", "")
    if xgb_job_arn:
        job_name = xgb_job_arn.split("/")[-1]
        job_desc = sm_client.describe_training_job(TrainingJobName=job_name)
        model_data_s3 = job_desc["ModelArtifacts"]["S3ModelArtifacts"]
        artifacts["training_job_name"] = job_name
        artifacts["model_data_s3"]     = model_data_s3
        print(f"  Training job   : {job_name}")
        print(f"  Model data S3  : {model_data_s3}")
    else:
        print("  ⚠  TrainXGBoost step not found in execution.")

    # ── Model Registry → model_package_arn ───────────────
    try:
        packages = sm_client.list_model_packages(
            ModelPackageGroupName="LiverDiseaseModelGroup",
            SortBy="CreationTime",
            SortOrder="Descending",
            MaxResults=1
        )["ModelPackageSummaryList"]
        if packages:
            pkg = packages[0]
            artifacts["model_package_arn"]  = pkg["ModelPackageArn"]
            artifacts["model_package_name"] = pkg["ModelPackageArn"]  # same value
            artifacts["model_package_ver"]  = pkg.get("ModelPackageVersion", "?")
            artifacts["approval_status"]    = pkg.get("ModelApprovalStatus", "?")
            print(f"  Model Package  : v{artifacts['model_package_ver']} "
                  f"[{artifacts['approval_status']}]")
            print(f"  Package ARN    : {artifacts['model_package_arn']}")
        else:
            print("  ⚠  No model packages found in LiverDiseaseModelGroup.")
    except Exception as e:
        print(f"  ⚠  Could not fetch model registry: {e}")

    # ── Processing outputs → baseline & test S3 URIs ─────
    proc_step = step_map.get("LiverDiseasePreprocessing", {})
    proc_job_arn = proc_step.get("Metadata", {}).get("ProcessingJob", {}).get("Arn", "")
    if proc_job_arn:
        job_name = proc_job_arn.split("/")[-1]
        proc_desc = sm_client.describe_processing_job(ProcessingJobName=job_name)
        for output in proc_desc["ProcessingOutputConfig"]["Outputs"]:
            name = output["OutputName"]
            uri  = output["S3Output"]["S3Uri"]
            if name == "baseline":
                artifacts["baseline_s3_uri"] = uri + "/baseline.csv"
                print(f"  Baseline S3    : {artifacts['baseline_s3_uri']}")
            elif name == "test":
                artifacts["test_s3_uri"] = uri
                print(f"  Test data S3   : {uri}")

    # ── Evaluation results → parsed metrics ──────────────
    eval_step = step_map.get("EvaluateModels", {})
    eval_job_arn = eval_step.get("Metadata", {}).get("ProcessingJob", {}).get("Arn", "")
    if eval_job_arn:
        job_name = eval_job_arn.split("/")[-1]
        eval_desc = sm_client.describe_processing_job(ProcessingJobName=job_name)
        for output in eval_desc["ProcessingOutputConfig"]["Outputs"]:
            if output["OutputName"] == "evaluation":
                eval_uri = output["S3Output"]["S3Uri"] + "/evaluation.json"
                try:
                    parts  = eval_uri.replace("s3://", "").split("/", 1)
                    obj    = s3_client.get_object(Bucket=parts[0], Key=parts[1])
                    eval_results = json.loads(obj["Body"].read())
                    artifacts["evaluation_results"] = eval_results
                    print(f"\n  ── Evaluation Results ──────────────────────────")
                    for model_key in ["xgboost", "random_forest"]:
                        m = eval_results.get(model_key, {})
                        if "error" not in m:
                            print(f"  {model_key:<15} acc={m.get('accuracy','?')}  "
                                  f"auc={m.get('auc','?')}  f1={m.get('f1','?')}")
                    print(f"  Best model     : {eval_results.get('best_model','?')}")
                except Exception as e:
                    print(f"  ⚠  Could not read evaluation.json: {e}")

    print("═" * 62)
    return artifacts


def approve_and_deploy(artifacts: dict,
                       endpoint_name: str = "liver-disease-endpoint") -> sagemaker.predictor.Predictor:
    """
    1. Auto-approves the registered model package
    2. Deploys it to a real-time endpoint
    Uses artifacts dict returned by extract_pipeline_artifacts().
    """
    # ── Step 1: Approve model ─────────────────────────────
    pkg_arn = artifacts.get("model_package_arn")
    if not pkg_arn:
        raise ValueError("model_package_arn not found in artifacts. "
                         "Did the RegisterLiverDiseaseModel step succeed?")

    sm_client.update_model_package(
        ModelPackageArn=pkg_arn,
        ModelApprovalStatus="Approved"
    )
    print(f"✅ Model approved: {pkg_arn}")

    # ── Step 2: Deploy endpoint ───────────────────────────
    model_data_s3 = artifacts.get("model_data_s3")
    if not model_data_s3:
        raise ValueError("model_data_s3 not found in artifacts.")

    predictor = deploy_model(
        model_data_s3=model_data_s3,
        endpoint_name=endpoint_name
    )
    return predictor

def cleanup(endpoint_name: str = "liver-disease-endpoint"):
    """Delete endpoint and monitoring schedules to avoid charges."""
    try:
        sm_client.delete_monitoring_schedule(MonitoringScheduleName="liver-data-quality-monitor")
        sm_client.delete_monitoring_schedule(MonitoringScheduleName="liver-model-quality-monitor")
        print("Monitoring schedules deleted.")
    except Exception as e:
        print(f"Monitor delete: {e}")

    try:
        sm_client.delete_endpoint(EndpointName=endpoint_name)
        print(f"Endpoint '{endpoint_name}' deleted.")
    except Exception as e:
        print(f"Endpoint delete: {e}")


# ─────────────────────────────────────────────────────────
# 17.  MAIN ORCHESTRATOR  (fully automatic end-to-end)
# ─────────────────────────────────────────────────────────

def main():
    print("=" * 62)
    print("  Liver Disease ML Pipeline – AWS SageMaker")
    print("=" * 62)

    # ── STEP 0: Save all helper scripts ───────────────────
    save_preprocessing_script()
    save_training_scripts()
    setup_cicd_files()

    # ── STEP 1: Upload raw data to S3 ─────────────────────
    raw_s3 = upload_raw_data("Liver_Patient_Dataset__LPD__train.csv")

    # ── STEP 2: Ingest into Feature Store ─────────────────
    df = pd.read_csv("Liver_Patient_Dataset__LPD__train.csv", encoding="latin-1").dropna()
    df.columns = df.columns.str.strip().str.replace("\xa0", "", regex=False)
    df.rename(columns={
        "Age of the patient":                   "Age",
        "Gender of the patient":                "Gender",
        "Total Bilirubin":                      "Total_Bilirubin",
        "Direct Bilirubin":                     "Direct_Bilirubin",
        "Alkphos Alkaline Phosphotase":         "Alkaline_Phosphotase",
        "Sgpt Alamine Aminotransferase":        "Alamine_Aminotransferase",
        "Sgot Aspartate Aminotransferase":      "Aspartate_Aminotransferase",
        "Total Protiens":                       "Total_Proteins",
        "ALB Albumin":                          "Albumin",
        "A/G Ratio Albumin and Globulin Ratio": "AG_Ratio",
        "Result":                               "Target"
    }, inplace=True)
    create_feature_group(df)

    # ── STEP 3: Build & run pipeline ──────────────────────
    build_pipeline()
    execution_arn, status = monitor_pipeline_execution("LiverDiseasePipeline")

    if status != "Succeeded":
        print(f"\n❌ Pipeline failed — stopping. Fix the reported step and re-run.")
        return

    # ── STEP 4: Auto-extract all ARNs / URIs from execution ─
    artifacts = extract_pipeline_artifacts(execution_arn)

    # ── STEP 5: Approve model + deploy endpoint ────────────
    ENDPOINT_NAME = "liver-disease-endpoint"
    try:
        predictor = approve_and_deploy(artifacts, endpoint_name=ENDPOINT_NAME)
    except Exception as e:
        print(f"⚠  Deployment skipped: {e}")
        predictor = None

    # ── STEP 6: CloudWatch dashboard ──────────────────────
    create_cloudwatch_dashboard(ENDPOINT_NAME)

    # ── STEP 7: Endpoint invocation examples ──────────────
    if predictor:
        try:
            invoke_endpoint_examples(ENDPOINT_NAME)
        except Exception as e:
            print(f"⚠  Endpoint invocation skipped: {e}")

    # ── STEP 8: Batch transform ───────────────────────────
    test_s3_uri = artifacts.get("test_s3_uri")
    if test_s3_uri and artifacts.get("training_job_name"):
        try:
            # ── Strip label column from test CSV for batch transform ──
            # The test CSV has label in col 0 + 13 features in cols 1-13.
            # Built-in XGBoost batch transform expects features ONLY (no label).
            import tempfile, io
            test_key = test_s3_uri.replace(f"s3://{BUCKET}/", "") + "/test.csv"
            obj       = s3_client.get_object(Bucket=BUCKET, Key=test_key)
            test_df   = pd.read_csv(io.BytesIO(obj["Body"].read()), header=None)
            features_only = test_df.iloc[:, 1:]   # drop label column 0

            # Upload features-only CSV to a separate S3 prefix
            features_key = f"{PREFIX}/batch-input/test_features.csv"
            buf = io.StringIO()
            features_only.to_csv(buf, index=False, header=False)
            s3_client.put_object(
                Bucket=BUCKET, Key=features_key,
                Body=buf.getvalue().encode("utf-8")
            )
            batch_input_uri = f"s3://{BUCKET}/{features_key}"
            print(f"  Features-only test data → {batch_input_uri}  "
                  f"({len(features_only)} rows × {features_only.shape[1]} cols)")

            # ── Create named model ────────────────────────────────────
            model_name = "liver-disease-xgboost"
            xgb_image  = sagemaker.image_uris.retrieve(
                framework="xgboost", region=region, version="1.5-1"
            )
            try:
                sm_client.create_model(
                    ModelName=model_name,
                    PrimaryContainer={
                        "Image":        xgb_image,
                        "ModelDataUrl": artifacts["model_data_s3"],
                    },
                    ExecutionRoleArn=role
                )
                print(f"  Model '{model_name}' created for batch transform.")
            except sm_client.exceptions.ResourceInUse:
                print(f"  Model '{model_name}' already exists — reusing.")

            run_batch_transform(
                model_name=model_name,
                test_s3_uri=batch_input_uri
            )
        except Exception as e:
            print(f"⚠  Batch transform skipped: {e}")
    else:
        print("⚠  Batch transform skipped: test_s3_uri or model not available.")

    # ── STEP 9: Setup monitoring ───────────────────────────
    baseline_s3_uri = artifacts.get("baseline_s3_uri")
    if baseline_s3_uri and predictor:
        try:
            setup_model_monitoring(ENDPOINT_NAME, baseline_s3_uri)
        except Exception as e:
            print(f"⚠  Monitoring setup skipped: {e}")
    else:
        print("⚠  Monitoring skipped: endpoint or baseline not available.")

    # ── STEP 10: Monitoring report ─────────────────────────
    try:
        generate_monitoring_report(ENDPOINT_NAME)
    except Exception as e:
        print(f"⚠  Monitoring report skipped: {e}")

    # ── DONE ───────────────────────────────────────────────
    print("\n" + "=" * 62)
    print("  ✅ PIPELINE COMPLETE — ARTIFACT SUMMARY")
    print("=" * 62)
    print(f"  Endpoint        : {ENDPOINT_NAME}")
    print(f"  Model data S3   : {artifacts.get('model_data_s3', 'N/A')}")
    print(f"  Model Package   : {artifacts.get('model_package_arn', 'N/A')}")
    print(f"  Baseline S3     : {artifacts.get('baseline_s3_uri', 'N/A')}")
    if artifacts.get("evaluation_results"):
        xgb = artifacts["evaluation_results"].get("xgboost", {})
        print(f"  XGBoost AUC     : {xgb.get('auc', 'N/A')}")
        print(f"  XGBoost Acc     : {xgb.get('accuracy', 'N/A')}")
    print("=" * 62)


if __name__ == "__main__":
    main()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Role    : arn:aws:iam::910617065862:role/service-role/AmazonSageMaker-ExecutionRole-20260217T114815
Bucket  : sagemaker-us-east-1-910617065862
Region  : us-east-1
  Liver Disease ML Pipeline – AWS SageMaker
Preprocessing script saved → preprocessing.py
Script saved → xgb_train.py
Script saved → rf_train.py
Script saved → evaluate.py
CI/CD files created.
Raw data uploaded → s3://sagemaker-us-east-1-910617065862/liver-disease/data/raw/Liver_Patient_Dataset__LPD__train.csv


/tmp/ipykernel_268/1404510094.py:211: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["event_time"] = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")


Feature group 'liver-disease-features' already exists (status: Created). Skipping creation.


INFO:sagemaker.feature_store.feature_group:Started ingesting index 10864 to 16296
INFO:sagemaker.feature_store.feature_group:Started ingesting index 21728 to 27158
INFO:sagemaker.feature_store.feature_group:Started ingesting index 0 to 5432
INFO:sagemaker.feature_store.feature_group:Started ingesting index 16296 to 21728
INFO:sagemaker.feature_store.feature_group:Started ingesting index 5432 to 10864
INFO:sagemaker.feature_store.feature_group:Successfully ingested row 10864 to 16296
INFO:sagemaker.feature_store.feature_group:Successfully ingested row 21728 to 27158
INFO:sagemaker.feature_store.feature_group:Successfully ingested row 16296 to 21728
INFO:sagemaker.feature_store.feature_group:Successfully ingested row 5432 to 10864
INFO:sagemaker.feature_store.feature_group:Successfully ingested row 0 to 5432
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to

Ingested 27158 records into Feature Store.
Preprocessing script saved → preprocessing.py
Old pipeline deleted — recreating with updated definition …


...

INFO:sagemaker:Creating endpoint-config with name liver-disease-endpoint


⚠  Deployment skipped: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:910617065862:endpoint-config/liver-disease-endpoint".
⚠  Could not create CloudWatch dashboard (IAM permission missing: cloudwatch:PutDashboard)
   Error: An error occurred (AccessDenied) when calling the PutDashboard operation: User: arn:aws:sts::910617065862:assumed-role/AmazonSageMaker-ExecutionRole-20260217T114815/SageMaker is not authorized to perform: cloudwatch:PutDashboard on resource: arn:aws:cloudwatch::910617065862:dashboard/LiverDisease-MLOps-Dashboard because no identity-based policy allows the cloudwatch:PutDashboard action

   ── To create the dashboard manually ─────────────────────────
   1. Open AWS Console → CloudWatch → Dashboards → Create dashboard
   2. Name it: LiverDisease-MLOps-Dashboard
   3. Add widgets for these metrics:
      • AWS/SageMaker → EndpointName=liver-dis

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


  Features-only test data → s3://sagemaker-us-east-1-910617065862/liver-disease/batch-input/test_features.csv  (5432 rows × 13 cols)
⚠  Batch transform skipped: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:us-east-1:910617065862:model/liver-disease-xgboost".
⚠  Monitoring skipped: endpoint or baseline not available.

═════════════════════════════════════════════════════════════════
  MODEL MONITORING REPORT
  Generated: 2026-02-20 15:49:25 UTC
═════════════════════════════════════════════════════════════════

── DATA QUALITY MONITOR ────────────────────────────
  ⚠  No violation report found yet (monitor may not have run).

── MODEL QUALITY MONITOR ───────────────────────────
  ⚠  No model quality statistics found yet.

── ENDPOINT TRAFFIC (last 1 hour via CloudWatch) ────
  Invocations (1h)    : N/A
  Avg Model Latency   : N/A
  5xx Errors          : 0
  Avg CPU Utilisation : N/A

── MODEL REGIS

/tmp/ipykernel_268/1404510094.py:1747: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"  Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")
/tmp/ipykernel_268/1404510094.py:1866: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now   = datetime.utcnow()


In [2]:
import boto3, json

runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

# One patient record — 13 features (NO label column):
# Age, Gender, TotalBili, DirectBili, AlkPhos, SGPT, SGOT, TotProt, Albumin, AGRatio, BiliRatio, EnzRatio, AgeGrp
patient = "65,1,0.7,0.1,187,16,18,6.8,3.3,0.90,0.14,0.89,2"

response = runtime.invoke_endpoint(
    EndpointName="liver-disease-endpoint",
    ContentType="text/csv",
    Body=patient
)

score = float(response["Body"].read().decode("utf-8"))
print(f"Raw score : {score:.4f}")
print(f"Prediction: {'liver_disease' if score >= 0.5 else 'no_disease'}")
print(f"Confidence: {score:.1%}")

Raw score : 0.8329
Prediction: liver_disease
Confidence: 83.3%
